In [1]:
import numpy as np 
import sympy as sy
import matplotlib.pyplot as plt
import tensorflow as tf 
import random
from IPython.core.interactiveshell import InteractiveShell
import warnings 
InteractiveShell.ast_node_interactive = "all"
warnings.filterwarnings("ignore")

2022-03-25 02:25:40.343971: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-25 02:25:40.343990: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Theory

Consider a system where:

\begin{equation}
h^t= f (h^{t-1},\theta)  
\end{equation}
\begin{equation}
\theta \text{: parameters shared across all time steps}
\end{equation}

That is, its state at time step t, is dependent only on a set a parameters and the previous state at t-1
<br>
<br>
Let the state of the system, h, also be depedent on an input at the respective time step, x:

\begin{equation}
h^t= f (h^{t-1},x^{t},\theta)  
\end{equation}

The state h now contains information about the entire past history of inputs, x.

Consider now a system that given the hidden state, h,produces an output o, for each time step. This output is passed to an activation function made to predict the target, y, at the respective time step.

\begin{equation}
o^t= g (h^{t},\theta')  
\end{equation}
\begin{equation}
\theta' \text{: a different set of parameters as $\theta$}
\end{equation}


We define now define $\theta$ and $\theta'$ as the weight matrices describing the relation between the input-to-hidden, hidden-to-hidden and hidden-to-output notes; $U$, $W$ and $V$:

\begin{equation}
z^t= (h^{t-1})^{T} W + (x^t)^{T}U +b 
\end{equation}

\begin{equation}
h^{t} = \phi(z^t)
\end{equation}

\begin{equation}
o^t = (h^{t})^{T}V + c
\end{equation}

Where $b$ and $c$ are biases, $\phi$ is an activation function. <br><br>
**Note**: matrices $U$, $W$ and $V$ are not indexed by time. 

Then for each time step, we have a sequential total loss up to time step $\tau$, $L^\tau$, defined as the difference between our prediction and the target, at each output, upto the time step $\tau$
<br>
<br>
Consider the task of multi-class classification. 
<br>
<br>
Consequently, the output activation function is the normalized expontential function, a.k.a the _softmax function_

\begin{equation}
L = \sum_{t=1}^{\tau} l\big(o^{t}\big)
\text{: Total loss upto time step $\tau$}  
\end{equation}

\begin{equation}
\hat{y}^t_i = \frac{\exp(o_i^t)}{\sum_{j}\exp(o_j^t)}
\text{: Softmax activation function for multi-class classification}
\end{equation}

\begin{equation}
l(o^{t}) = \sum_{m=0}^{M-1}y_{m}^{t}(o^t) \log\Bigg(\frac{1}{\hat{y}_{m}^{t}(o^t)}\Bigg)
\text{: M categorical cross entropy for predictions at time step $t$}
\end{equation}

The optimization process differs from standard back-propagation (like descirbed for a vanilla feedforward network). Usng the above assumptions, I will go through the derivation analogous optimization process for recurrent networks;

### Back propagation through time

Per example loss w.r.t to the output element $o_i$ at time $t$; $o_i^{t}$

\begin{equation}
\frac{\partial{L}}{\partial o_{i}^{t}} = \frac{\partial{L}}{\partial{l(o_i^t)}} \frac{\partial{l(o_i^t)}}{\partial o_{i}^{t}}
\end{equation}
Note that:
\begin{equation}
 \frac{\partial{L}}{\partial{l(o_i^t)}} = 1
\end{equation}
and that:
\begin{equation}
 \frac{\partial{l(o_i^t)}}{\partial o_{i}^{t}}
\end{equation}
is the derivative of the categorical cross-entropy
\begin{equation}
\boxed{
 \frac{\partial{l(o_i^t)}}{\partial o_{i}^{t}} = - \sum_{j} \frac{y_j^{t}}{\hat{y}_j^{t}}\frac{\partial{\hat{y}^{t}_j}}{\partial{o_i^{t}}} } - [1]
\end{equation}
The softmax functions is:
 \begin{equation}
 \hat{y}^t_i = \frac{\exp(o_i^t)}{\sum_{j}\exp(o_j^t)}
\end{equation}
Taking its derivative gives:
\begin{equation}
\boxed{
    \frac{\partial{\hat{y}^{t}_i}}{\partial{o_j^{t}}} = \hat{y}^{t}_{i} \Big(\delta_{ij} - \hat{y}^{t}_{j}\Big)
}- [2]
\end{equation}
_look at the different cases to see why this is true_ i.e. $i=j$ and $i \neq j$
<br><br>
Lets sub [2] into [1]:
 \begin{equation}
 \frac{\partial{l(o_i^t)}}{\partial o_{i}^{t}} = \sum_{j} \frac{y_j^{t}}{\hat{y}_j^{t}} \hat{y}^{t}_{i} \Big(\hat{y}^{t}_{j} - \delta_{ij}\Big)
\end{equation}
Which becomes:
\begin{equation}
  \frac{\partial{l(o_i^t)}}{\partial o_{i}^{t}}  = \hat{y}^t_{i} - y^t_i
\end{equation}
Summarising this a vector:
\begin{equation}
  \boxed {
    \nabla_{o^{t}} L  = \hat{y}^t - y^t
  }
  \end{equation}

Next, lets calculate the gradient on the internel nodes $h^t$. I will use vector notation here on out. I.e. $h_i^{t}$ becomes $h^t$

In [ ]:
class RNN:
    def __init__(self, input_dim, hidden_dim=128):
        # network variables 
        self.idim = input_dim
        self.hdim = hidden_dim
        # initialise weights 
        self.U = np.random.uniform(- np.sqrt(1./self.idim),
                                     np.sqrt(1./self.idim),
                                    (self.idim, self.hdim) )
        self.V = np.random.uniform( -np.sqrt(1./self.hdim),
                                     np.sqrt(1./self.hdim), 
                                    (self.hdim,self.idim))
        self.W = np.random.uniform( -np.sqrt(1./self.hdim),
                                     np.sqrt(1./self.hdim), 
                                    (self.hdim,self.hdim))

        self.b = np.zeros(self.hdim)
        self.c = np.zeros(self.idim)
    

    def softmax(self,x):
        '''Note that this is a numerically stable version of softmax. We substract the max value from all elements.
        Overflow of a single element, or underflow of all elements,  will render the output usless.
        
        subtracting max leaves only non-positive values ---> no overflow 
        at least one element = 0 ---> no vanishing denominator (underflow is some enteries is okay) 
         '''
        xt = np.exp(x-np.max(x))
        return xt / np.sum(xt)

    def forward(self,x):
        # Single example pass forward
        T = len(x)
        h = np.zeros((T,self.hdim))
        o = np.zeros((T,self.idim))

        for t in range(T):
            ###
            h[t] = self.U[x[t], :] + self.b
            ####
            if t > 1:
                h[t] += self.W @ h[t] + self.c
            h[t] = np.tanh(h[t])
            o[t] = self.softmax( self.V @ h[t] + self.c)
        return (o,t)

    def backward(self, x, y, clip=None):
        T = len(x)
        o,h = self.forward(x)
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)
        dLdb = np.zeros(self.b.shape)
        dLdc = np.zeros(self.c.shape)

        # dL/do
        delta_o = o
        delta_o[np.arange(len(y)), y] -= 1.
        # dL/dh
        delta_h = np.zeros((T, self.hdim))
        
        for t in reversed(range(T)):
            delta_h[t] = self.V @ delta_o[t,:]
            if t < T-1:
                delta_h[t] = ( self.W @ np.diag(1-h[t+1]**2) ) @ delta_h[t+1]
        for t in range(T):
            dLdc += delta_o[t,:]
            dLdb += (1-h[t]**2) * delta_h[t,:]
            dLdV += h[t,:]@ delta_o[t,:].T 
            if t > 0 :
                dLdW += ( h[t-1,:] @ delta_h[t,:].T )@np.diag(1-h[t]**2)
            xm = np.zeros((self.idim))
            xm[x] = 1. 
            dLdU += xm @ delta_h[t,:].T @ np.diag(1-h[t]**2)

        if clip is not None:
            dLdb = np.clip(dLdb, -clip, clip)
            dLdc = np.clip(dLdc, -clip, clip)
            dLdV = np.clip(dLdV, -clip, clip)
            dLdW = np.clip(dLdW, -clip, clip)
            dLdU = np.clip(dLdU, -clip, clip)

        return (dLdU, dLdV, dLdW, dLdb, dLdc)

    def step(self,x,y,lr=0.01):
        dLdU, dLdV, dLdW, dLdb, dLdc = self.backward(x,y)
        self.U -= lr * dLdU
        self.V -= lr * dLdV
        self.W -= lr * dLdW 
        self.b -= lr * dLdb 
        self.c -= lr * dLdc 
    

    def Loss(self, x,y):
        o,h = self.forward(x)
        return -np.sum(o[np.arange(len(y)), y])



In [6]:
np.diag(np.arange(3,7))[0]

array([3, 0, 0, 0])